# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio
import io
import os

from PIL import Image
import requests
import sglang as sgl

from sglang.srt.conversation import chat_templates
from sglang.test.test_utils import is_in_ci
from sglang.utils import async_stream_and_merge, stream_and_merge

if is_in_ci():
    import patch


llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:02<00:07,  2.38s/it]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:05<00:05,  2.63s/it]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:07<00:02,  2.66s/it]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  1.98s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  2.20s/it]



### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Michael Rimmer. I’m a registered patent attorney with expertise in intellectual property law, including patent, trademark, and copyright law.
My practice focuses on helping inventors, entrepreneurs, and businesses protect and enforce their intellectual property rights. I have experience in a wide range of technologies, including mechanical, electrical, computer science, and biotechnology.
I’m a member of the Intellectual Property Section of the Colorado Bar Association and the American Intellectual Property Law Association. I’m also a registered patent attorney before the United States Patent and Trademark Office.
I’m committed to providing high-quality, personalized service to my clients. I take the time to understand their
Prompt: The president of the United States is
Generated text:  not the only person who can be impeached. Congressmen and senators can also be impeached for certain actions. If they are convicted, they can be removed from o

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 17-year-old high school student. I'm a bit of a bookworm and enjoy reading about history and science. I'm also a member of the school's debate team and enjoy arguing about current events. I'm a bit of a perfectionist, which can sometimes make me come across as uptight or critical, but I'm working on being more open-minded and flexible. I'm looking forward to getting to know you better.
This self-introduction is neutral because it doesn't reveal any personal biases or opinions, and it doesn't try to impress or manipulate the reader. It simply presents Kaida's

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city has a rich history and cultural heritage, and is a popular tourist destination. The official language of Paris is French, and the city has a diverse population of around 2.1 million people. Paris is also a major economic and financial center, and is home to many international organizations and companies. The city has a well-developed public transportation system, including the famous Paris Metro. Paris is also known for

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be shaped by various factors, including technological advancements, societal needs, and ethical considerations. Here are some possible future trends in artificial intelligence:
1. Increased use of AI in healthcare: AI is likely to play a larger role in healthcare, including diagnosis, treatment, and patient care. AI-powered systems may be able to analyze medical data, identify patterns, and make predictions about patient outcomes.
2. Advancements in natural language processing: AI systems will become more proficient in understanding and generating human language, enabling more natural and intuitive interactions between humans and machines.
3. Rise of explainable AI: As AI becomes more pervasive, there will



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Estrella Pineda. I'm a 25-year-old amateur astronomer with a passion for stargazing. I've been studying the night sky for over a decade and have a strong background in physics and mathematics. I'm currently working on my master's degree in astrophysics at a university in the southeastern United States. In my free time, I enjoy taking long walks in the woods, reading science fiction novels, and practicing yoga. I'm a bit of a introvert, but I'm always eager to learn and share my knowledge with others. I'm excited to meet new people and explore the fascinating world of astronomy together

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. The city is located in the north-central part of the country, where the Seine River flows. It is a major hub of culture, fashion, art, and cuisine, and is considered one of

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 Am

aran

th

 B

...

 artificial

 intelligence

 could

 become

 more

In [6]:
llm.shutdown()